In [ ]:
import os

# Load API keys from a txt file
def load_keys_from_file(filepath="keys.txt"):
    if not os.path.exists(filepath):
        raise FileNotFoundError(f"Key file not found: {filepath}")

    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            if "=" not in line:
                continue
            key, val = line.split("=", 1)
            os.environ[key.strip()] = val.strip()
        

load_keys_from_file("api.txt")

API_KEY_BLS = (os.getenv("BLS_API_KEY") or "").strip() or None
API_KEY_FRED = (os.getenv("FRED_API_KEY") or "").strip() or None


In [ ]:
import os
import requests
import pandas as pd
from datetime import date


# Config / constants
BLS_API_URL = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
FRED_SERIES_ID = "FEDFUNDS"  # Effective Federal Funds Rate (percent)
FRED_OBS_URL = "https://api.stlouisfed.org/fred/series/observations"

# BLS series IDs (seasonally adjusted)
SERIES = {
    "cpi_all_items_sa": "CUSR0000SA0",      # CPI-U, all items, SA
    "unemployment_rate_sa": "LNS14000000",  # Unemployment rate, SA
}

# BLS Data Pull (with 10-year chunking)
def fetch_bls_timeseries(series_ids, start_year, end_year=None, api_key=None):
    """Fetch BLS time series, chunking by the 10-year API limit."""
    if end_year is None:
        end_year = date.today().year

    out = {sid: [] for sid in series_ids}
    for chunk_start in range(start_year, end_year + 1, 10):
        chunk_end = min(chunk_start + 9, end_year)
        payload = {
            "seriesid": series_ids,
            "startyear": str(chunk_start),
            "endyear": str(chunk_end),
        }
        if api_key:
            payload["registrationkey"] = api_key

        r = requests.post(BLS_API_URL, json=payload, timeout=60)
        r.raise_for_status()
        data = r.json()

        if data.get("status") != "REQUEST_SUCCEEDED":
            msg = data.get("message")
            if isinstance(msg, list):
                msg = "; ".join(map(str, msg))
            raise RuntimeError(f"BLS API error: {msg}")

        for s in data["Results"]["series"]:
            sid = s["seriesID"]
            out[sid].extend(s["data"])
    return out

def bls_to_dataframe(series_dict, rename=None, drop_annual=True):
    """Convert BLS timeseries dict to tidy monthly DataFrame."""
    frames = []
    for sid, rows in series_dict.items():
        recs = []
        for row in rows:
            if row.get("period", "").upper() == "M13" and drop_annual:
                continue
            year = int(row["year"])
            month = int(row["period"][1:])
            recs.append({
                "date": pd.Timestamp(year=year, month=month, day=1),
                sid: float(row["value"]),
            })
        if recs:
            df = pd.DataFrame(recs).sort_values("date").set_index("date")
            if rename and sid in rename:
                df = df.rename(columns={sid: rename[sid]})
            frames.append(df)
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, axis=1).sort_index()

# FRED Data Pull
def fetch_fred_monthly(series_id, start_date, end_date, api_key=None):
    """
    Fetch a FRED series as monthly averages between start_date and end_date (YYYY-MM-DD).
    Returns a DataFrame indexed by first-of-month with a single float column.
    """
    params = {
        "series_id": series_id,
        "file_type": "json",
        "observation_start": start_date,
        "observation_end": end_date,
        "frequency": "m",             # aggregate to monthly
        "aggregation_method": "avg",  # monthly average (for daily series)
        "units": "lin",               # level
    }
    if api_key:
        params["api_key"] = api_key

    r = requests.get(FRED_OBS_URL, params=params, timeout=60)
    r.raise_for_status()
    data = r.json()

    if "observations" not in data:
        raise RuntimeError(f"FRED API error: {data.get('error_message', 'unknown error')}")

    recs = []
    for o in data["observations"]:
        val = o.get("value")
        if val == "." or val is None:
            continue
        # first day of the month
        d = pd.to_datetime(o["date"]).to_period("M").to_timestamp()
        recs.append({"date": d, "fed_funds_rate_pct": float(val)})

    if not recs:
        return pd.DataFrame(columns=["fed_funds_rate_pct"]).astype({"fed_funds_rate_pct": "float64"})

    df = pd.DataFrame(recs).drop_duplicates("date").set_index("date").sort_index()
    return df


# Main

if __name__ == "__main__":
    today = date.today()
    END_YEAR = today.year
    START_YEAR = END_YEAR - 25  # last 25 years (rolling)

    # Read keys from environment variables; treat blanks as None
    API_KEY_BLS = (os.getenv("BLS_API_KEY") or "").strip() or None       # optional
    API_KEY_FRED = (os.getenv("FRED_API_KEY") or "").strip() or None     # required by FRED API

    if not API_KEY_BLS:
        print("Warning: no BLS key found; you may hit the anonymous quota.")
    if not API_KEY_FRED:
        print("Warning: no FRED key found; FRED request may fail.")

    # --- BLS ---
    series_ids = list(SERIES.values())
    pretty_names = {
        SERIES["cpi_all_items_sa"]: "cpi_sa_index",
        SERIES["unemployment_rate_sa"]: "unemp_rate_sa_pct",
    }
    raw = fetch_bls_timeseries(
        series_ids=series_ids,
        start_year=START_YEAR,
        end_year=END_YEAR,
        api_key=API_KEY_BLS,
    )
    df_bls = bls_to_dataframe(raw, rename=pretty_names)

    # Trim to last 25 years by month
    cutoff = pd.Timestamp(today.year - 25, today.month, 1)
    df_bls = df_bls[df_bls.index >= cutoff]

    # Add YoY CPI % change (optional)
    if "cpi_sa_index" in df_bls.columns:
        df_bls["cpi_yoy_pct"] = df_bls["cpi_sa_index"].pct_change(12) * 100

    # FRED (Fed funds rate, monthly avg) 
    start_iso = f"{cutoff.year}-{cutoff.month:02d}-01"
    end_iso = f"{today.year}-{today.month:02d}-01"
    df_fed = fetch_fred_monthly(FRED_SERIES_ID, start_iso, end_iso, api_key=API_KEY_FRED)

    # Merge & Save 
    df = df_bls.join(df_fed, how="left")

    df.to_csv("macro_last25yrs_bls_fred.csv")


            cpi_sa_index  unemp_rate_sa_pct  cpi_yoy_pct  fed_funds_rate_pct
date                                                                        
2024-09-01       314.851                4.1     2.432541                5.13
2024-10-01       315.564                4.1     2.571403                4.83
2024-11-01       316.449                4.2     2.714168                4.64
2024-12-01       317.603                4.1     2.872366                4.48
2025-01-01       319.086                4.0     2.999413                4.33
2025-02-01       319.775                4.1     2.814270                4.33
2025-03-01       319.615                4.2     2.405585                4.33
2025-04-01       320.321                4.2     2.333747                4.33
2025-05-01       320.580                4.2     2.375934                4.33
2025-06-01       321.500                4.1     2.672683                4.33
2025-07-01       322.132                4.2     2.731801                4.33